In [23]:
# Import necessary libraries
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
# 1. Load a pre-trained sentence transformer model for encoding documents
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

In [25]:
# Example documents (you can replace this with real documents or a corpus)
documents = [
    "Quantum computing is a paradigm of computation that leverages quantum mechanical phenomena such as superposition and entanglement to process information.",
    "Unlike classical bits, quantum bits (qubits) can exist in multiple states simultaneously, enabling certain computations to be performed more efficiently.",
    "This technology has the potential to revolutionize fields such as cryptography, optimization, material science, and drug discovery."
]

In [26]:
# 2. Encode the documents into embedding using sentence transformer
document_embeddings = sentence_model.encode(documents, convert_to_tensor=True)

In [27]:
# 3. Create a FAISS index for fast retrieval
dimension = document_embeddings.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(dimension)      # Use L2 distance for similiarity search
index.add(np.array(document_embeddings))  # Add embeddings to FAISS index

In [ ]:
# 4. Load a pre-trained T5 model and tokenizer for text generation
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [29]:
# 5. Define a function to retrieve the most relavant documents based on a query
def retrieve_documents(query, top_k=1):
    query_embedding = sentence_model.encode([query], convert_to_tensor=True)

    # Perform retrieval
    _, indices = index.search(np.array(query_embedding), top_k)

    # Retrieve the top k most relavant documents
    retrieve_docs = [documents[i] for i in indices[0]]
    return retrieve_docs

In [30]:
# 6. Define a function for Summarization using RAG-based Approach
def rag_summarization(query):
    # Retrieve the most relavant documents for the query
    retrieved_docs = retrieve_documents(query)

    # Combine retrieved documents into one string to form the context
    context = "".join(retrieved_docs)

    # Prepare the input text for the model (query + context)
    input_text = f"Summarize: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate a summary using T5 model
    generated_ids = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)

    # Decode the generated output to text (summary)
    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return output

In [31]:
# 7. Test the Summarzation RAG system
query = "Tell me about Quantum Computing"
summary = rag_summarization(query)
print(summary)

Summarize: Quantum computing is a paradigm of computation that leverages quantum mechanical phenomena such as superposition and entanglement to process information.
